In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

#import time
#import argparse
#import multiprocessing


from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
#from IPython.display import display

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')

C:\ProgramData\Anaconda3\envs\tensorflow_final\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\envs\tensorflow_final\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\envs\tensorflow_final\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\envs\tensorflow

In [2]:
# This is needed to display the images.
%matplotlib inline

In [3]:
from object_detection.utils import label_map_util
from object_detection.utils import ops as utils_ops
from object_detection.utils import visualization_utils as vis_util

In [4]:
MODEL_NAME = 'inference_graph/ssd_mobilenet_v2_coco_82610'
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = os.path.join('annotations', 'labelmap-all.pbtxt')

In [5]:
NUM_CLASSES = 5

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES)
category_index = label_map_util.create_category_index(categories)

In [6]:
def run_inference_for_single_image(image, graph):
    if 'detection_masks' in tensor_dict:
        # Processamento de uma única imagem
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # É necessário atualizar a imagem para converter a máscara das coordenadas da caixa em coordenadas da imagem e ajustar o tamanho da imagem.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
    image_tensor = tf.compat.v1.get_default_graph().get_tensor_by_name('image_tensor:0')
    #print(image_tensor.eval, "\n")

    # Run inference
    output_dict = sess.run(tensor_dict,
                            feed_dict={image_tensor: np.expand_dims(image, 0)})
    #print(output_dict, "\n")
    # all outputs are float32 numpy arrays, so convert types as appropriate
    output_dict['num_detections'] = int(output_dict['num_detections'][0])
    output_dict['detection_classes'] = output_dict[
        'detection_classes'][0].astype(np.uint8)
    output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
    output_dict['detection_scores'] = output_dict['detection_scores'][0]
    if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

In [7]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.compat.v1.GraphDef()
  with tf.compat.v2.io.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [8]:
#import glob

import pandas as pd

#targetPattern = r"CSGO_images\test\*.jpg"
#arquivos = glob.glob(targetPattern)

pasta = r"CSGO_images//test//"
fileExt = r".jpg"
fileExt_1 = r".JPG"
arquivos  = [_ for _ in os.listdir(pasta) if _.endswith(fileExt) or _.endswith(fileExt_1)]
arquivos

['1outa1.jpg',
 '1outa100.jpg',
 '1outa112.jpg',
 '1outa153.jpg',
 '1outa167.jpg',
 '1outa174.jpg',
 '1outa180.jpg',
 '1outa185.jpg',
 '1outa20.jpg',
 '1outa211.jpg',
 '1outa227.jpg',
 '1outa238.jpg',
 '1outa239.jpg',
 '1outa265.jpg',
 '1outa27.jpg',
 '1outa270.jpg',
 '1outa288.jpg',
 '1outa293.jpg',
 '1outa300.jpg',
 '1outa32.jpg',
 '1outa323.jpg',
 '1outa358.jpg',
 '1outa376.jpg',
 '1outa381.jpg',
 '1outa410.jpg',
 '1outa421.jpg',
 '1outa44.jpg',
 '1outa444.jpg',
 '1outa447.jpg',
 '1outa448.jpg',
 '1outa452.jpg',
 '1outa454.jpg',
 '1outa460.jpg',
 '1outa462.jpg',
 '1outa513.jpg',
 '1outa524.jpg',
 '1outa539.jpg',
 '1outa557.jpg',
 '1outa56.jpg',
 '1outa573.jpg',
 '1outa580.jpg',
 '1outa584.jpg',
 '1outa592.jpg',
 '1outa605.jpg',
 '1outa608.jpg',
 '1outa61.jpg',
 '1outa615.jpg',
 '1outa620.jpg',
 '1outa680.jpg',
 '1outa710.jpg',
 '1outa711.jpg',
 '1outa716.jpg',
 '1outa724.jpg',
 '1outa735.jpg',
 '1outa740.jpg',
 '1outa749.jpg',
 '1outa75.jpg',
 '1outa751.jpg',
 '1outa772.jpg',
 '1out

In [9]:
test_csv = pd.read_csv('test_labels_1.csv')
arr = np.array(test_csv)
arr

array([['1outa1.jpg', 2, 0.38888888899999996, 0.484375,
        0.7517361109999999, 0.59765625],
       ['1outa100.jpg', 2, 0.42708333299999995, 0.436523438,
        0.7795138890000001, 0.6171875],
       ['1outa112.jpg', 2, 0.43576388899999996, 0.47265625, 0.84375,
        0.5859375],
       ...,
       ['IMG_9365.JPG', 0, nan, nan, nan, nan],
       ['IMG_9366.JPG', 0, nan, nan, nan, nan],
       ['IMG_9367.JPG', 0, nan, nan, nan, nan]], dtype=object)

In [10]:
import cv2
previsoes = []
l = 0

Verdadeiro_positivo = 0
Falso_positivo = 0
Falso_negativo = 0
Verdadeiro_negativo = 0

for w in arquivos:
    image_np = cv2.imread(pasta + w)
    print("step:", l)
    l = l + 1 
    try:
        with detection_graph.as_default():
                with tf.compat.v1.Session() as sess:
                    
                    # Get handles to input and output tensors
                    ops = tf.compat.v1.get_default_graph().get_operations()
                    all_tensor_names = {output.name for op in ops for output in op.outputs}
                    tensor_dict = {}
                    
                    for key in [
                      'num_detections', 'detection_boxes', 'detection_scores',
                      'detection_classes', 'detection_masks'
                    ]:
                        tensor_name = key + ':0'
                        if tensor_name in all_tensor_names:
                            tensor_dict[key] = tf.compat.v1.get_default_graph().get_tensor_by_name(
                          tensor_name)
                            #print(tensor_dict[key], "\n")

                    #ret, image_np = cap.read()
                    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
                    image_np_expanded = np.expand_dims(image_np, axis=0)
                    #print("image_np_expanded", "\n")
                    # Actual detection.
                    output_dict = run_inference_for_single_image(image_np, detection_graph)
                    #print(output_dict, "\n")

                    test_result = tf.greater(output_dict['detection_scores'], [0.50])
                    j = 0

                    linha = []
                    previsoes_2 = []
                    linha.append(w)
                    
                    
                    for i in test_result.eval():
                        if i == True:
                            linha = []
                            linha.append(w)
                            linha.append(output_dict['detection_classes'][j])
                            linha.append(output_dict['detection_boxes'][j][0])
                            linha.append(output_dict['detection_boxes'][j][1])
                            linha.append(output_dict['detection_boxes'][j][2])
                            linha.append(output_dict['detection_boxes'][j][3])
                            #print(linha_1, "\n")
                            
                            #linha.append(linha_1)
                            #linha.append(linha_2)

                            previsoes.append(linha)
                            previsoes_2.append(linha)
                            print(linha, "\n")
                            
                        j = j + 1
                        
                    index = np.argwhere(arr == w)
                    print("index")
                    print("1", index)
                    print(len(index))
                    print("arr1", arr[index[0][0]][1])

                    if len(previsoes_2) != 0:       
                        
                        matrix_resultado = []
                        q = []
                        n = 0
                        for i in index:
                            k = []
                            m = 0
                            for g in previsoes_2:
                                Erro_cord = []
                                #print("if2", arr[i[0]][1] == g[1], "identificação de classe", arr[i[0]][1], g[1])
                                if arr[i[0]][1] == g[1]:
                                    Erro_cord.append(arr[i[0]][2] - g[2])
                                    Erro_cord.append(arr[i[0]][3] - g[3])
                                    Erro_cord.append(arr[i[0]][4] - g[4])
                                    Erro_cord.append(arr[i[0]][5] - g[5])
                                    erro_1 = tf.greater(Erro_cord, [0.05])
                                    erro_2 = tf.less(Erro_cord, [-0.05])
                                    with tf.compat.v1.Session() as sess:
                                        if any(o == True for o in erro_1.eval()) or any(o == True for o in erro_2.eval()):
                                            #print("Coordenada errada")
                                            k.append(False)
                                            
                                        else:                           
                                            #print("acetou")
                                            k.append(True)
                                            
                                else:
                                    #print("Classe errada")
                                    k.append(False)
                                m = m + 1        
                            q.append(k)
                            print("k", k)
                                
                            n = n + 1
                        print("q", q)
                        print("m", m)
                        print("n", n)
                        
                        df = pd.DataFrame(q)
                        for f in range(0, m, 1):
                            print("F1", f)
                            c = np.all(df[f] == False)
                            print("Coluna", f, c)
                            if c:
                                Falso_positivo = Falso_positivo + 1
                        
                        for f in range(0, n, 1):
                            print("F2", f)
                            c = np.any(q[f])
                            print("Linha", f, c)
                            if c:
                                Verdadeiro_positivo = Verdadeiro_positivo + 1
                            else:
                                Falso_negativo = Falso_negativo + 1
                    
                    else:
                        if arr[index[0][0]][1] == 0:
                            Verdadeiro_negativo = Verdadeiro_negativo + 1
                    
                        else:                        
                            Falso_negativo = Falso_negativo + 1
                            #print("não encontrou nada", "Falso_negativo", Falso_negativo)
                    
                    print("Falso_negativo", Falso_negativo)
                    print("Verdadeiro_positivo", Verdadeiro_positivo)
                    print("Falso_positivo", Falso_positivo)
                    print("Verdadeiro_negativo", Verdadeiro_negativo)  
                    
                    # Visualization of the results of a detection.
                    vis_util.visualize_boxes_and_labels_on_image_array(
                        image_np,
                        output_dict['detection_boxes'],
                        output_dict['detection_classes'],
                        output_dict['detection_scores'],
                        category_index,
                        instance_masks=output_dict.get('detection_masks'),
                        use_normalized_coordinates=True,
                        line_thickness=3)
                    #cv2.imshow('object_detection', cv2.resize(image_np, (800, 600)))
                    #while True:
                        #if cv2.waitKey(25) & 0xFF == ord('q'):
                            #cv2.destroyAllWindows()
                            #break

    except Exception as e:
        print(e)
        #cap.release()

print("Falso_negativo", Falso_negativo)
print("Verdadeiro_positivo", Verdadeiro_positivo)
print("Falso_positivo", Falso_positivo)
print("Verdadeiro_negativo", Verdadeiro_negativo)  


step: 0
['1outa1.jpg', 2, 0.38585007, 0.477614, 0.78440917, 0.6003558] 

index
1 [[0 0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 0
Verdadeiro_positivo 1
Falso_positivo 0
Verdadeiro_negativo 0
step: 1
['1outa100.jpg', 2, 0.4250577, 0.43580908, 0.78757674, 0.6130876] 

index
1 [[1 0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 0
Verdadeiro_positivo 2
Falso_positivo 0
Verdadeiro_negativo 0
step: 2
['1outa112.jpg', 2, 0.42974794, 0.46918368, 0.857754, 0.5915911] 

index
1 [[2 0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 0
Verdadeiro_positivo 3
Falso_positivo 0
Verdadeiro_negativo 0
step: 3
['1outa153.jpg', 2, 0.4526282, 0.385947, 0.7272869, 0.58673495] 

index
1 [[3 0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 0
Verdadeiro_positivo 4
Falso_positivo 0
Verdadeiro_negativo 0
step: 4
['1outa167.jpg', 2

['1outa460.jpg', 2, 0.3672806, 0.41459718, 0.69546854, 0.59304017] 

index
1 [[32  0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 0
Verdadeiro_positivo 33
Falso_positivo 4
Verdadeiro_negativo 0
step: 33
['1outa462.jpg', 2, 0.3596686, 0.4106723, 0.6899756, 0.5987216] 

index
1 [[33  0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 0
Verdadeiro_positivo 34
Falso_positivo 4
Verdadeiro_negativo 0
step: 34
['1outa513.jpg', 2, 0.50889385, 0.44733214, 0.7862916, 0.64619696] 

['1outa513.jpg', 2, 0.39483693, 0.0019125044, 0.88153505, 0.28968713] 

index
1 [[34  0]]
1
arr1 2
k [True, False]
q [[True, False]]
m 2
n 1
F1 0
Coluna 0 False
F1 1
Coluna 1 True
F2 0
Linha 0 True
Falso_negativo 0
Verdadeiro_positivo 35
Falso_positivo 5
Verdadeiro_negativo 0
step: 35
['1outa524.jpg', 2, 0.46660244, 0.4316724, 0.79286873, 0.64116186] 

index
1 [[35  0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0

['1outa782.jpg', 2, 0.334031, 0.4416831, 0.69659877, 0.5756628] 

index
1 [[61  0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 1
Verdadeiro_positivo 61
Falso_positivo 16
Verdadeiro_negativo 0
step: 62
['1outa789.jpg', 2, 0.31696743, 0.45621124, 0.7336406, 0.6176489] 

index
1 [[62  0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 1
Verdadeiro_positivo 62
Falso_positivo 16
Verdadeiro_negativo 0
step: 63
['1outa791.jpg', 2, 0.34863806, 0.48144552, 0.76811373, 0.60371584] 

index
1 [[63  0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 1
Verdadeiro_positivo 63
Falso_positivo 16
Verdadeiro_negativo 0
step: 64
['1outa804.jpg', 2, 0.37314937, 0.43335053, 0.73336446, 0.59480375] 

index
1 [[64  0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 1
Verdadeiro_positivo 64
Falso_positivo 16
Verdadeiro_negativo 0
step: 6

step: 91
['1outb201.jpg', 1, 0.45874578, 0.44872242, 0.77203673, 0.62213105] 

index
1 [[91  0]]
1
arr1 1
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 2
Verdadeiro_positivo 90
Falso_positivo 24
Verdadeiro_negativo 0
step: 92
['1outb213.jpg', 1, 0.44003534, 0.4074867, 0.7518561, 0.6235378] 

index
1 [[92  0]]
1
arr1 1
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 2
Verdadeiro_positivo 91
Falso_positivo 24
Verdadeiro_negativo 0
step: 93
['1outb231.jpg', 1, 0.43402314, 0.4265223, 0.76399827, 0.61710423] 

index
1 [[93  0]]
1
arr1 1
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 2
Verdadeiro_positivo 92
Falso_positivo 24
Verdadeiro_negativo 0
step: 94
['1outb235.jpg', 1, 0.43439925, 0.42491895, 0.75890756, 0.6322381] 

index
1 [[94  0]]
1
arr1 1
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 2
Verdadeiro_positivo 93
Falso_positivo 24
Verdadeiro_negativo

['1outb543.jpg', 1, 0.46002585, 0.42539847, 0.7207168, 0.6102488] 

index
1 [[123   0]]
1
arr1 1
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 2
Verdadeiro_positivo 122
Falso_positivo 26
Verdadeiro_negativo 0
step: 124
['1outb545.jpg', 1, 0.4561112, 0.42039412, 0.73397434, 0.6256736] 

index
1 [[124   0]]
1
arr1 1
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 2
Verdadeiro_positivo 123
Falso_positivo 26
Verdadeiro_negativo 0
step: 125
['1outb548.jpg', 1, 0.45042294, 0.4077072, 0.74190074, 0.6163422] 

index
1 [[125   0]]
1
arr1 1
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 2
Verdadeiro_positivo 124
Falso_positivo 26
Verdadeiro_negativo 0
step: 126
['1outb560.jpg', 1, 0.43799382, 0.4339673, 0.7771835, 0.59571975] 

index
1 [[126   0]]
1
arr1 1
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 2
Verdadeiro_positivo 125
Falso_positivo 26
Verdadeiro_negat

...

q [[True, False]]
m 2
n 1
F1 0
Coluna 0 False
F1 1
Coluna 1 True
F2 0
Linha 0 True
Falso_negativo 11
Verdadeiro_positivo 211
Falso_positivo 36
Verdadeiro_negativo 0
step: 222
['1outc683.jpg', 5, 0.37785187, 0.51795626, 0.99371517, 0.8233551] 

index
1 [[222   0]]
1
arr1 5
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 11
Verdadeiro_positivo 212
Falso_positivo 36
Verdadeiro_negativo 0
step: 223
['1outc684.jpg', 5, 0.36181486, 0.517724, 0.99346936, 0.82410866] 

index
1 [[223   0]]
1
arr1 5
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 11
Verdadeiro_positivo 213
Falso_positivo 36
Verdadeiro_negativo 0
step: 224
['1outc692.jpg', 5, 0.35556346, 0.43677938, 0.9649673, 0.8634919] 

index
1 [[224   0]]
1
arr1 5
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 11
Verdadeiro_positivo 214
Falso_positivo 36
Verdadeiro_negativo 0
step: 225
['1outc700.jpg', 5, 0.37034917, 0.44742107, 1.0, 0.85487926

k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 12
Verdadeiro_positivo 242
Falso_positivo 38
Verdadeiro_negativo 0
step: 254
['1outd38.jpg', 4, 0.5227892, 0.5552483, 0.76093286, 0.7960976] 

index
1 [[254   0]]
1
arr1 4
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 12
Verdadeiro_positivo 243
Falso_positivo 38
Verdadeiro_negativo 0
step: 255
index
1 [[255   0]]
1
arr1 4
Falso_negativo 13
Verdadeiro_positivo 243
Falso_positivo 38
Verdadeiro_negativo 0
step: 256
['1outd41.jpg', 4, 0.50875735, 0.54110014, 0.7789297, 0.7805158] 

index
1 [[256   0]]
1
arr1 4
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 13
Verdadeiro_positivo 244
Falso_positivo 38
Verdadeiro_negativo 0
step: 257
['1outd44.jpg', 4, 0.49693233, 0.52349794, 0.78636986, 0.7874061] 

index
1 [[257   0]]
1
arr1 4
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 13
Verdadeiro_positivo 245
Falso_pos

step: 292
['1oute330.jpg', 3, 0.10844955, 0.28951842, 0.9889889, 0.74543685] 

index
1 [[292   0]]
1
arr1 3
k [False]
q [[False]]
m 1
n 1
F1 0
Coluna 0 True
F2 0
Linha 0 False
Falso_negativo 29
Verdadeiro_positivo 264
Falso_positivo 45
Verdadeiro_negativo 0
step: 293
['1oute332.jpg', 3, 0.18396041, 0.30676723, 0.961684, 0.74689984] 

index
1 [[293   0]]
1
arr1 3
k [False]
q [[False]]
m 1
n 1
F1 0
Coluna 0 True
F2 0
Linha 0 False
Falso_negativo 30
Verdadeiro_positivo 264
Falso_positivo 46
Verdadeiro_negativo 0
step: 294
['1oute342.jpg', 3, 0.4892169, 0.4078982, 0.9722594, 0.8565891] 

index
1 [[294   0]]
1
arr1 3
k [False]
q [[False]]
m 1
n 1
F1 0
Coluna 0 True
F2 0
Linha 0 False
Falso_negativo 31
Verdadeiro_positivo 264
Falso_positivo 47
Verdadeiro_negativo 0
step: 295
index
1 [[295   0]]
1
arr1 3
Falso_negativo 32
Verdadeiro_positivo 264
Falso_positivo 47
Verdadeiro_negativo 0
step: 296
index
1 [[296   0]]
1
arr1 3
Falso_negativo 33
Verdadeiro_positivo 264
Falso_positivo 47
Verdadeiro

k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 44
Verdadeiro_positivo 286
Falso_positivo 51
Verdadeiro_negativo 0
step: 330
['IMG_8875.JPG', 2, 0.43269435, 0.30455464, 0.81440365, 0.6276102] 

index
1 [[330   0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 44
Verdadeiro_positivo 287
Falso_positivo 51
Verdadeiro_negativo 0
step: 331
['IMG_8884.JPG', 2, 0.48361093, 0.4648804, 0.79061526, 0.6181759] 

index
1 [[331   0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 44
Verdadeiro_positivo 288
Falso_positivo 51
Verdadeiro_negativo 0
step: 332
['IMG_8894.JPG', 2, 0.33988726, 0.4378853, 0.8244785, 0.72160685] 

index
1 [[332   0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 44
Verdadeiro_positivo 289
Falso_positivo 51
Verdadeiro_negativo 0
step: 333
['IMG_8913.JPG', 2, 0.38305613, 0.46552342, 0.68181396, 0.65306765] 

index


step: 369
['IMG_9360.JPG', 4, 0.0126540065, 0.010906875, 0.99294573, 0.9124975] 

index
1 [[370   0]]
1
arr1 0
k [False]
q [[False]]
m 1
n 1
F1 0
Coluna 0 True
F2 0
Linha 0 False
Falso_negativo 58
Verdadeiro_positivo 296
Falso_positivo 69
Verdadeiro_negativo 17
step: 370
index
1 [[371   0]]
1
arr1 0
Falso_negativo 58
Verdadeiro_positivo 296
Falso_positivo 69
Verdadeiro_negativo 18
step: 371
['IMG_9362.JPG', 1, 0.37208524, 0.3618976, 0.532337, 0.55337363] 

index
1 [[372   0]]
1
arr1 0
k [False]
q [[False]]
m 1
n 1
F1 0
Coluna 0 True
F2 0
Linha 0 False
Falso_negativo 59
Verdadeiro_positivo 296
Falso_positivo 70
Verdadeiro_negativo 18
step: 372
index
1 [[373   0]]
1
arr1 0
Falso_negativo 59
Verdadeiro_positivo 296
Falso_positivo 70
Verdadeiro_negativo 19
step: 373
['IMG_9364.JPG', 5, 0.0, 0.29961035, 0.6233387, 0.63615453] 

index
1 [[374   0]]
1
arr1 0
k [False]
q [[False]]
m 1
n 1
F1 0
Coluna 0 True
F2 0
Linha 0 False
Falso_negativo 60
Verdadeiro_positivo 296
Falso_positivo 71
Verdadei

In [11]:
presisao = Verdadeiro_positivo/(Verdadeiro_positivo+Falso_positivo)
presisao

0.8

In [13]:
seletividade = Verdadeiro_positivo/(Verdadeiro_positivo+Falso_negativo)
seletividade

0.8245125348189415

In [15]:
especificidade = Verdadeiro_negativo/(Verdadeiro_negativo+Falso_positivo)
especificidade

0.20430107526881722

In [17]:
acuracidade = (Verdadeiro_positivo+Verdadeiro_negativo)/(Verdadeiro_positivo+Verdadeiro_negativo+Falso_negativo+Falso_positivo)
acuracidade

0.6969026548672567